In [2]:
import pandas as pd
import numpy as np


#### the following codes are similar in part 1 & 2 in previous assignment, hence many explanatory notes left out

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
df[0].head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
# It is confirmed df[0] is the table we seek

pdf=df[0] # use variable pdf to work


# code to extract info from pdf.iloc

new_df=[]; nblist=''
for i in range(pdf.shape[0]):
    
    item=pdf.iloc[i]
    pcode=item[0]; borough=item[1]; nbhood=item[2]; 

    if borough =='Not assigned' and nbhood == 'Not assigned':
        pcode=''
    
    if pcode != '':
        
        if borough != 'Not assigned' and nbhood =='Not assigned':
            nbhood=borough
            
        if i > 0 :
            if pcode == pdf.iloc[i-1,0]:
                nblist += ', ' + nbhood
            else:
                nblist=nbhood
        else:  
             nblist=nbhood 
                
        if i < pdf.shape[0]:
            if pcode != pdf.iloc[i+1,0]:
                item[2] = nblist
                new_df.append(item)
                nblist=''
        else:
            if nblist !='':
                item[2] = nblist
                new_df.append(item)
                nblist=''
                    

In [6]:
new_pdf=pd.DataFrame(new_df)
new_pdf.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Harbourfront, Regent Park"
7,M6A,North York,"Lawrence Heights, Lawrence Manor"
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,"Rouge, Malvern"
14,M3B,North York,Don Mills North
16,M4B,East York,"Woodbine Gardens, Parkview Hill"
18,M5B,Downtown Toronto,"Ryerson, Garden District"


In [10]:
new_pdf.shape

(103, 3)

In [7]:
ndf = pd.read_csv('http://cocl.us/Geospatial_data')
ndf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### merging the new_pdf with ndf to form merge_ndf dataframe

In [13]:
merge_pdf = pd.merge(left=new_pdf, right=ndf, left_on='Postcode', right_on='Postal Code')

In [17]:
merge_copy=merge_pdf.copy()
merge_ndf=merge_copy.drop(['Postal Code'], axis=1)
merge_ndf.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [19]:
merge_ndf.shape

(103, 5)